In [215]:
# Reading a Dataset
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
import seaborn as sns
shapefile=gpd.read_file('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/Datasets/telangana_shapefile.geojson')
fires_df=pd.read_csv('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/Datasets/telangana_fires.csv')

import warnings
warnings.filterwarnings("ignore")

In [2]:
shapefile.head()

,GID_0,NAME_0,GID_1,NAME_1,VARNAME_1,NL_NAME_1,TYPE_1,ENGTYPE_1,CC_1,HASC_1,geometry
0,IND,India,IND.32_1,Telangana,None,None,State,State,None,IN.TG,"MULTIPOLYGON (((79.21147 16.36281, 79.22324 16..."


In [3]:
shapefile.explore(column='NAME_1',cmap='Set2',tiles='Stamen Terrain')

In [4]:
fires_df.shape

(18785, 16)

In [5]:
fires_df.head()

,fireID,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,0,18.3777,80.5314,318.6,1.0,1.0,02/01/15,802,Aqua,MODIS,69,6.03,302.6,7.7,D,0
1,1,17.5898,80.9893,315.4,1.2,1.1,04/01/15,750,Aqua,MODIS,59,6.03,299.5,8.8,D,0
2,2,17.6028,81.1612,311.8,1.2,1.1,05/01/15,527,Terra,MODIS,46,6.03,299.8,6.0,D,0
3,3,17.5241,80.3135,317.0,1.1,1.0,05/01/15,527,Terra,MODIS,69,6.03,302.0,9.3,D,0
4,4,17.7571,80.3404,317.2,2.5,1.5,05/01/15,833,Aqua,MODIS,53,6.03,300.0,29.3,D,0


In [6]:
fires_df.instrument.value_counts()

MODIS    18785
Name: instrument, dtype: int64

In [7]:
fires_df.type.value_counts()

0    18413
2      372
Name: type, dtype: int64

In [8]:
fires_df.confidence.value_counts()

65    554
68    536
67    531
66    531
61    525
     ... 
8       4
6       4
9       4
11      3
5       1
Name: confidence, Length: 96, dtype: int64

In [9]:
fires_df.loc[fires_df.confidence>50].shape

(14311, 16)

In [10]:
fires_df.dtypes

fireID          int64
latitude      float64
longitude     float64
brightness    float64
scan          float64
track         float64
acq_date       object
acq_time        int64
satellite      object
instrument     object
confidence      int64
version       float64
bright_t31    float64
frp           float64
daynight       object
type            int64
dtype: object

In [11]:
india=gpd.read_file('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/AvinashMahech/gadm40_IND_Level3_shp/gadm40_IND_3.shp')
india.head()

,ID_0,COUNTRY,NAME_1,NL_NAME_1,NAME_2,NL_NAME_2,ID_3,NAME_3,VARNAME_3,NL_NAME_3,TYPE_3,ENGTYPE_3,CC_3,HASC_3,geometry
0,IND,India,Andaman and Nicobar,None,Nicobar Islands,None,IND.1.1.1_1,n.a. ( 2304),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((93.78988 6.85201, 93.79015 6.8..."
1,IND,India,Andaman and Nicobar,None,North and Middle Andaman,None,IND.1.2.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((92.84441 12.14969, 92.84466 12..."
2,IND,India,Andaman and Nicobar,None,South Andaman,None,IND.1.3.1_1,n.a. ( 2178),None,None,Taluk,Taluk,None,None,"MULTIPOLYGON (((92.52111 10.89694, 92.52306 10..."
3,IND,India,Andhra Pradesh,None,Anantapur,None,IND.2.1.1_1,Anantapur,None,None,Taluk,Taluk,None,None,"POLYGON ((77.84469 14.58576, 77.83913 14.58792..."
4,IND,India,Andhra Pradesh,None,Anantapur,None,IND.2.1.2_1,Dharmavaram,None,None,Taluk,Taluk,None,None,"POLYGON ((77.35452 14.52155, 77.35507 14.53332..."


In [12]:
telangana=india[india['NAME_1']=='Telangana']

In [13]:
telangana.shape

(72, 15)

In [14]:
telangana.area.sum()

9.776575621421403

In [15]:
shapefile.area.sum()

9.776575599803248

In [16]:
telangana.explore(column='NAME_1',cmap='Set2',tiles='Stamen Terrain',
                 tooltip=['NAME_3','NAME_2'])

In [17]:
telangana.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
fires_df['adm_name']=0
fires_df['year']=0

for i in range(len(fires_df)):
    fires_df.loc[i,'year']='20'+fires_df.iloc[i]['acq_date'].split('/')[-1]
    point=Point(fires_df.iloc[i]['longitude'],fires_df.iloc[i]['latitude'])
    for j in range(len(telangana)):
        polygon=telangana.iloc[j]['geometry']
        if point.within(polygon) and fires_df.iloc[i]['confidence']>45: # considering whose confodence greater than 45 percentage
            fires_df.loc[i,'adm_name']=telangana.iloc[j]['NAME_3']

In [19]:
fires_df=fires_df.loc[fires_df.adm_name!=0]

In [59]:
df=pd.DataFrame()
df['fireCount']=fires_df.groupby(['adm_name','year'])['adm_name'].count()
df=df.reset_index()

In [61]:
df.head()

,adm_name,year,fireCount
0,Achampet,2015,111
1,Achampet,2016,71
2,Achampet,2017,27
3,Achampet,2018,111
4,Achampet,2019,100


In [217]:
import plotly.express as px


fig = px.bar(df, x="adm_name", y="fireCount", color="year", title="Fire detecttion")
fig.show()

In [67]:
df.to_json('D:/Projects/UNDP/Data4Policy/Geospatial Data Science Internship/AvinashMahech/output1.json',orient='index')

In [159]:
geo_df=df.copy()
geo_df['geometry']=0

for i in range(len(df)):
    geo_df.loc[i,'geometry']=telangana.loc[telangana.NAME_3==geo_df.iloc[i]['adm_name']].geometry.values[0]

In [160]:
geo_df

,adm_name,year,fireCount,geometry
0,Achampet,2015,111,"POLYGON ((79.21147155800008 16.36281013500019,..."
1,Achampet,2016,71,"POLYGON ((79.21147155800008 16.36281013500019,..."
2,Achampet,2017,27,"POLYGON ((79.21147155800008 16.36281013500019,..."
3,Achampet,2018,111,"POLYGON ((79.21147155800008 16.36281013500019,..."
4,Achampet,2019,100,"POLYGON ((79.21147155800008 16.36281013500019,..."
...,...,...,...,...
487,n.a. ( 1728),2017,3,"POLYGON ((78.30213165200018 17.14622116200002,..."
488,n.a. ( 1728),2018,7,"POLYGON ((78.30213165200018 17.14622116200002,..."
489,n.a. ( 1728),2019,2,"POLYGON ((78.30213165200018 17.14622116200002,..."
490,n.a. ( 1728),2020,5,"POLYGON ((78.30213165200018 17.14622116200002,..."


In [161]:
geo_df = gpd.GeoDataFrame(geo_df, geometry=geo_df.geometry)
geo_df.crs={'init':'epsg:4326'}
geo_df.to_crs(telangana.crs,inplace=True)
geo_df.year=pd.to_datetime(geo_df.year)

In [176]:
geo_df.drop(geo_df.tail(13).index,inplace=True)